<a href="https://colab.research.google.com/github/pserebrennikov/3rd-year-project/blob/master/4_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 4 - Regularization techniques
### Course on Optimization for Machine Learning - Dr. F. Ballarin
### Master Degree in Data Analytics for Business, Catholic University of the Sacred Heart, Milano

In this notebook we discuss regularization techniques for regression problems.

In [ ]:
import typing

In [ ]:
import numpy as np
import plotly.colors
import plotly.graph_objects as go
import plotly.subplots

## Exercise 4.1

Consider a scalar feature $x$ in the domain domain $I = [-1, 1]$ and the observation $y$ defined as
$$y = g(x) + \xi,$$
where the (deterministic) function $g$ is defined as
$$g(x) = \cos(1.5 \pi x)$$
and $\xi$ is a white Gaussian noise with zero mean and variance $\sigma^2 = 0.01$.

1. Generate a training dataset $\boldsymbol{x}_{\text{train}} \in \mathbb{R}^{200}$ and a test dataset $\boldsymbol{x}_{\text{test}} \in \mathbb{R}^{50}$ using features drawn from a uniform distribution on $I$. Generate the corresponding train and test datasets ($\boldsymbol{y}_{\text{train}} \in \mathbb{R}^{200}$ and $\boldsymbol{y}_{\text{test}} \in \mathbb{R}^{50}$, respectively) for the observation.

*Solution*:
> We first define the deterministic function $g(x)$ in a Python function `g(x)`. Note that calling such Python function for a vector of features (e.g., all rows in a dataset) gives a vector of the elementwise evaluation of $g$.

In [ ]:
def g(x: np.ndarray) -> np.ndarray:
    """Evaluate g(x)."""
    return np.cos(1.5 * np.pi * x)

> We set a seed for reproducibility, and generate the required datasets using [`np.random.uniform`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.uniform.html) and [`np.random.normal`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html).

In [ ]:
np.random.seed(41)
x_train = np.random.uniform(-1, 1, size=30)
x_test = np.random.uniform(-1, 1, size=50)
y_train = g(x_train) + np.random.normal(0, np.sqrt(0.01), size=30)
y_test = g(x_test) + np.random.normal(0, np.sqrt(0.01), size=50)

> We prepare of plot of the two datasets, on top of the deterministic function $g$.

In [ ]:
x_plot = np.linspace(-1, 1, 500)
y_plot = g(x_plot)

In [ ]:
fig = plotly.subplots.make_subplots(rows=2, cols=1, vertical_spacing=0.05)
fig.add_scatter(
    x=x_train, y=y_train,
    marker=dict(color=plotly.colors.qualitative.Set1[0], size=6),
    mode="markers", name="training dataset",
    row=1, col=1
)
fig.add_scatter(
    x=x_test, y=y_test,
    marker=dict(color=plotly.colors.qualitative.Set1[1], size=6),
    mode="markers", name="test dataset",
    row=2, col=1
)
fig.add_scatter(
    x=x_plot, y=y_plot,
    line=dict(color=plotly.colors.qualitative.Set1[2], width=2),
    mode="lines", name="g(x)",
    row=1, col=1
)
fig.add_scatter(
    x=x_plot, y=y_plot,
    line=dict(color=plotly.colors.qualitative.Set1[2], width=2),
    mode="lines", name="g(x)", showlegend=False,
    row=2, col=1
)
fig.update_layout(
    title="Training dataset (top) and test dataset (bottom)",
    width=640, height=640, autosize=False
)
fig.show()

2. Implement the evaluation of the empirical risk associated to a polynomial regression via least squares, as well as its gradient.

*Solution*:
> Similarly to previous exercises on logistic regression, we first start by implementing the prediction function $\hat{y}(x; \boldsymbol{w}) = \sum_{i = 0}^{n - 1} w^{(i)} x^i$ associated to a polynomial regression. Note that the value of $n$, which controlas the maximum polynomial degree $n - 1$, will be specified later.

In [ ]:
def y_hat(x_j: float, w: np.ndarray) -> float:
    """Evaluate the prediction function associated to a polynomial regression."""
    return sum(w[i] * x_j**i for i in range(w.shape[0]))

> The least squares loss and its derivatives are:
$$\ell(x, y; \boldsymbol{w}) = \left(\sum_{i = 0}^{n - 1} w^{(i)} x^i - y\right)^2.$$
$$\nabla_\boldsymbol{w} \ell(x, y; \boldsymbol{w}) = 2 \left(\sum_{i = 0}^{n - 1} w^{(i)} x^i - y\right) \begin{bmatrix}1\\x\\x^2\\\dots\\x^{n-1}\end{bmatrix}.$$

In [ ]:
def least_squares_loss(x_j: float, y_j: float, w: np.ndarray) -> float:
    """Evaluate the least squares loss."""
    return (y_hat(x_j, w) - y_j)**2

In [ ]:
def grad_least_squares_loss(x_j: float, y_j: float, w: np.ndarray) -> np.ndarray:
    """Evaluate the gradient of the least squares loss."""
    powers = np.arange(w.shape[0])
    return 2 * (y_hat(x_j, w) - y_j) * x_j**powers

> The empirical risk is the obtained summing over all elements of the training dataset.

In [ ]:
def f_ex_4_1(w: np.ndarray) -> float:
    """Evaluate the empirical risk."""
    m_train = x_train.shape[0]
    return 1 / m_train * sum(least_squares_loss(x_j, y_j, w) for (x_j, y_j) in zip(x_train, y_train))

In [ ]:
def grad_f_ex_4_1(w: np.ndarray) -> np.ndarray:
    """Evaluate the gradient of the empirical risk."""
    m_train = x_train.shape[0]
    return 1 / m_train * sum(grad_least_squares_loss(x_j, y_j, w) for (x_j, y_j) in zip(x_train, y_train))

3. Implement a Python function for the minimization of a function $f$ using BFGS with constant step length. Such function should:
   * take as input the function $f$, its gradient $\nabla f$, the constant step length $\alpha$, the tolerance $\varepsilon$ and $K_{\max}$ for the stopping criterion, the initial conditions $\boldsymbol{w}_{0}$ and $h_0$;
   * return as outputs the optimization variable iterations $\{\boldsymbol{w}_k\}_k$, the corresponding function values $\{f(\boldsymbol{w}_k)\}_k$ and gradients $\{\nabla f(\boldsymbol{w}_k)\}_k$.
 
   Use the following initialization:
   * initial condition $\boldsymbol{w}_{0}$ for the optimization variable, and
   * define the initial approximation of the inverse of the hessian matrix as $\boldsymbol{H}_0 = h_0 \boldsymbol{I}$, where $h_0$ is the last input argument.
 
   Use the following stopping criteria:
   * the main stopping criterion is based on the norm of the gradient. Continue the iterations while such norm is above $\varepsilon$;
   * allow a maximum number of iterations $K_{\max}$ of iterations, after which the method should terminate with a warning.
 
*Solution*:
> We have already implemented BFGS in Tutorial 2. The main difference is that here (as typically done in machine learning) we require an initialization for $\boldsymbol{H}_0$, while in Tutorial 2 we assumed it was possible to evaluate the hessian matrix exactly as the initial iteration.

In [ ]:
def bfgs(
    f: typing.Callable, grad_f: typing.Callable, alpha: float, epsilon: float, maxit: int, w_0: np.ndarray,
    h_0: float
) -> typing.Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Run BFGS method with constant step length.

    Parameters
    ----------
    f, grad_f : Python function
        callable evaluating the cost function and its gradient, respectively.
    alpha : float
        constant step length.
    c_1, c_2 : float
        constants of the backtracking algorithm.
    epsilon : float
        tolerance for the stopping criterion on the error on the norm of the gradient of the cost.
    maxit : int
        maximum number of allowed iterations.
    w_0 : 1d numpy array
        numpy array containing the initial condition.
    h_0 : float
        diagonal scaling factor that defines the initial approximation of the inverse of the hessian matrix.

    Returns
    -------
    2d numpy array
        history of the optimization variables iterations.
    1d numpy array
        history of the cost function values.
    2d numpy array
        history of the gradient of the cost function.
    """
    # Prepare lists collecting the required outputs over the iterations
    all_w = [w_0]
    all_f = [f(w_0)]
    all_grad_f = [grad_f(w_0)]

    # Prepare iteration counter
    k = 0

    # Prepare initial approximation of inverse of the hessian: take the identity matrix
    I = np.eye(w_0.shape[0])
    inv_hess_f_k = h_0 * I

    # Use the norm of the gradient as stopping criterion.
    while np.linalg.norm(all_grad_f[k]) > epsilon:
        w_k = all_w[k]
        grad_f_k = all_grad_f[k]

        # Compute w_{k+1}
        w_k_plus_1 = w_k - alpha * np.dot(inv_hess_f_k, grad_f_k)

        # Update required outputs
        all_w.append(w_k_plus_1)
        all_f.append(f(w_k_plus_1))
        all_grad_f.append(grad_f(w_k_plus_1))

        # Update approximation of inverse of the hessian
        y_k = all_grad_f[k + 1] - all_grad_f[k]
        s_k = all_w[k + 1] - all_w[k]
        rho_k = 1 / np.dot(y_k, s_k)
        inv_hess_f_k = (
            np.dot(np.dot(I - rho_k * np.outer(s_k, y_k), inv_hess_f_k), I - rho_k * np.outer(y_k, s_k))
            + rho_k * np.outer(s_k, s_k)
        )

        # Increment iteration counter
        k += 1

        # Bail out if exceeded allowed number of iterations
        if k >= maxit:
            print("WARNING: BFGS method exceeded number of allowed iterations")
            break

    # For convenience we transform the outputs into numpy array before returning
    return np.array(all_w), np.array(all_f), np.array(all_grad_f)

4. Choose $\varepsilon = 10^{-8}$, $\boldsymbol{w}_0 = \boldsymbol{0}$ and $h_0 = 1$. Obtain the best fit polynomial for $n$ between 1 and 25 by running BFGS. Comment on the quality of the fit when $n$ increases.

*Solution*:
> We run the `bfgs` function for all required values of $n$.

In [ ]:
all_w = dict()
all_f = dict()
all_grad_f = dict()
for n in range(1, 26):
    all_w[n], all_f[n], all_grad_f[n] = bfgs(f_ex_4_1, grad_f_ex_4_1, 1, 1e-8, 1000, np.zeros(n), 1)

> We next create the following plot to get a graphical intuition of the concepts of underfitting and overfitting. The plot has two rows, corresponding to the training set (top) and the test set (bottom), and four columns, each column corresponding to a value of $n \in \{3, 7, 13, 25\}$.

In [ ]:
n_plot = [3, 7, 13, 25]
fig = plotly.subplots.make_subplots(rows=2, cols=4, vertical_spacing=0.05, horizontal_spacing=0.05)
for (col, n) in enumerate(n_plot):
    fig.add_scatter(
        x=x_train, y=y_train,
        marker=dict(color=plotly.colors.qualitative.Set1[0], size=6),
        mode="markers", name="training dataset",
        row=1, col=col + 1, showlegend=(col == 0)
    )
    fig.add_scatter(
        x=x_test, y=y_test,
        marker=dict(color=plotly.colors.qualitative.Set1[1], size=6),
        mode="markers", name="test dataset",
        row=2, col=col + 1, showlegend=(col == 0)
    )
    fig.add_scatter(
        x=x_plot, y=y_hat(x_plot, all_w[n][-1]),
        line=dict(color=plotly.colors.qualitative.Set1[6], width=2),
        mode="lines", name="y_hat(x)",
        row=1, col=col + 1, showlegend=(col == 0)
    )
    fig.add_scatter(
        x=x_plot, y=y_hat(x_plot, all_w[n][-1]),
        line=dict(color=plotly.colors.qualitative.Set1[6], width=2),
        mode="lines", name="y_hat(x)",
        row=2, col=col + 1, showlegend=False
    )
fig.update_layout(
    title=(
        "Training dataset (top) and test dataset (bottom). "
        + "Increasing n = " + str(n_plot) + " (left to right)"
    ), width=2 * 512, height=512, autosize=False
)
# fig.update_yaxes(range=[-1.2, 1.2])
fig.show()

> * From the first column we see that the generated model with $n = 3$ is underfitting the data: the prediction curve is not able to describe especially the peak near $x = 0$ (which is underestimated by the prediction), and at least one of the regions with low values (either $x \approx \pm 2/3$.
> * The second column seems to represent a nice fit. The overall pattern of the data is well reproduced both the training and the test datasets.
> * The third and fourth columns show increasing overfitting. Uncomment the code line that restricts the vertical axis to realize that the polynomial is trying to pass through as many data point as possible (especially close to the left and right boundaries of the domain $I$), but this is causing oscillations of larger and larger magnitude when $n$ increases. Since these oscillations try to match the training dataset, but *cannot* match the unseen test dataset, they will lead in general to worse predictions.
>
> A more precise quantification of whether a model is underfitting or overfitting can be obtained introducing the concept of RMSE (root mean square error), defined as
$$\text{RMSE}(\boldsymbol{x}, \boldsymbol{y}) = \sqrt{\frac{1}{m} \sum_{j = 1}^{m} (\hat{y}(x_j) - y_j)^2}.$$
> Upon computing the RMSE on both train and test datasets, the following cases may occur:
> * $\text{RMSE}(\boldsymbol{x}_{\text{train}}, \boldsymbol{y}_{\text{train}}) \gg 0$ and $\text{RMSE}(\boldsymbol{x}_{\text{test}}, \boldsymbol{y}_{\text{test}}) \gg 0$: this is a case of underfitting, where the model is either not trained properly or is not rich enough to describe the phenomenon that generated the data. In this exercise, underfitting happens when $n < 5$
> * $\text{RMSE}(\boldsymbol{x}_{\text{train}}, \boldsymbol{y}_{\text{train}}) \approx 0$ and $\text{RMSE}(\boldsymbol{x}_{\text{test}}, \boldsymbol{y}_{\text{test}}) \approx 0$: this is the desired case, in which a model correctly describes the data both in the training and in the test sets. In this exercise, this corresponds to $5 \leq n \leq 8$
> * $\text{RMSE}(\boldsymbol{x}_{\text{train}}, \boldsymbol{y}_{\text{train}}) \approx 0$ and $\text{RMSE}(\boldsymbol{x}_{\text{test}}, \boldsymbol{y}_{\text{test}}) \gg 0$: this is the case of overfitting, in which the model has memorized the training data very well, but has poor generalization capabilities when provided with unseen test data. In this exercise, this corresponds to $n > 8$.
> * (the remaining case, $\text{RMSE}(\boldsymbol{x}_{\text{train}}, \boldsymbol{y}_{\text{train}}) \gg 0$ and $\text{RMSE}(\boldsymbol{x}_{\text{test}}, \boldsymbol{y}_{\text{test}}) \approx 0$ never occurs in practice)

In [ ]:
RMSE_train = np.array([
    np.sqrt(1 / x_train.shape[0] * np.sum((y_hat(x_train, all_w[n][-1]) - y_train)**2))
    for n in range(1, 26)
])
RMSE_test = np.array([
    np.sqrt(1 / x_test.shape[0] * np.sum((y_hat(x_test, all_w[n][-1]) - y_test)**2))
    for n in range(1, 26)
])

In [ ]:
fig = go.Figure()
fig.add_scatter(x=np.arange(RMSE_train.shape[0]) + 1, y=RMSE_train, name="Training set")
fig.add_scatter(x=np.arange(RMSE_test.shape[0]) + 1, y=RMSE_test, name="Test set")
fig.update_layout(title="RMSE")
fig.update_yaxes(type="log", exponentformat="power")
fig.show()

> It is clear that underfitting is quite simple to detect while training the model, while overfitting can be detected only by querying data that are yet to be seen. There are validation techniques available, but often require putting apart some of the training data into a validation dataset. In alternative, is there a way to *change* the optimization problem to prevent overfitting?

## Exercise 4.2

The US National Centers for Environmental Information has collected weather data at the Raleigh Durham International Airport. The dataset contains a long list of fields, starting from the date
* `date`: the date the following observations were collected,

and associated numerical observations:
* `temperaturemin`: the minimum temperature (in Fahrenheit),
* `temperaturemax`: the maximum temperature (in Fahrenheit),
* `precipitation`: the total amount of precipitation (in inches),
* `snowfall`: the total amount of snow precipitation (in inches),
* `snowdepth`: the maximum depth of snow (in inches),
* `avgwindspeed`: the average wind speed (in miles per hour),
* `fastest2minwinddir`: the direction of fastest 2-minute wind (in degrees),
* `fastest2minwindspeed`: the fastest 2-minute wind speed (in miles per hour),
* `fastest5secwinddir`: the direction of fastest 5-second wind (in degrees),
* `fastest5secwindspeed`: the fastest 5-second wind speed (in miles per hour),

or boolean observations:
* `fog`: the text `Present` is used if there has been fog during the day; if blank, it means that no fog was registered during that day,
* `fogheavy`: the text `Present` is used if there has been heavy fog during the day; if blank, it means fog was either not registered during that day, or was not classified as heavy,
* `mist`: the text `Present` is used if there has been mist during the day; if blank, it means that no mist was registered during that day,
* `rain`: the text `Present` is used if there has been rain during the day; if blank, it means that no rain was registered during that day,
* `fogground`: the text `Present` is used if there has been fog at ground level during the day; if blank, it means fog was either not registered during that day, or was not at ground level,
* `ice`: the text `Present` is used if there have been ice pellets, sleet, snow pellets, or small hail during the day; if blank, it means that no ice was registered during that day,
* `glaze`: the text `Present` is used if there have been glaze or rime during the day; if blank, it means that no glaze was registered during that day,
* `drizzle`: the text `Present` is used if there has been drizzle during the day; if blank, it means that no drizzle was registered during that day,
* `snow`:  the text `Present` is used if there have been snow, snow pellets, snow grains, or ice crystals during the day; if blank, it means that no snow was registered during that day,
* `freezingrain`: the text `Present` is used if there has been freezing rain during the day; if blank, it means that either no rain was registered during that day, or it was not classified as freezing rain,
* `smokehaze`: the text `Present` is used if there has been smoke during the day; if blank, it means that no smoke was registered during that day,
* `thunder`: the text `Present` is used if there have been thunders during the day; if blank, it means that no thunder was registered during that day,
* `highwind`: the text `Present` is used if there has been high or damaging wind during the day; if blank, it means that either no wind was registered during that day, or any registered wind was not particularly high,
* `hail`: the text `Present` is used if there has been hail during the day; if blank, it means that no hail was registered during that day,
* `blowingsnow`: the text `Present` is used if there has been blowing or drifting snow during the day; if blank, it means that either no snow was registered during that day, or any registered snow was not classified as blowing,
* `dust`: the text `Present` is used if there have been dust, volcanic ash, blowing dust or blowing sand during the day; if blank, it means that no dust was registered during that day,
* `freezingfog`: the text `Present` is used if there has been freezing fog during the day; if blank, it means fog was either not registered during that day, or was not classified as freezing.

The goal of this exercise is to predict the minimum temperature of any given day as a function of the remaining columns of the dataset.

1. Load the dataset of historical data from [its website](`https://data.townofcary.org/explore/dataset/rdu-weather-history/information/`)

*Solution*:
> We use `pandas` to load the dataset.

In [ ]:
import os

In [ ]:
import pandas as pd

In [ ]:
if os.path.isfile("data/rdu-weather-history.csv"):
    csv_path = "data/rdu-weather-history.csv"
else:
    # csv_path = "https://data.townofcary.org/explore/dataset/rdu-weather-history/download/?format=csv"
    csv_path = (
        "https://dmf.unicatt.it/~fball/public/optimization_for_machine_learning"
        + "/rdu-weather-history.csv"
    )
df = pd.read_csv(csv_path, parse_dates=["date"], sep=";")

> We see that the dataset has thousands of rows and 28 columns.

In [ ]:
df

> The boolean columns contain either the string `Present`, or `NaN`. We replace them, respectively, with 1 and 0.

In [ ]:
bool_cols = [
    "fog", "fogheavy", "mist", "rain", "fogground", "ice", "glaze", "drizzle", "snow", "freezingrain",
    "smokehaze", "thunder", "highwind", "hail", "blowingsnow", "dust", "freezingfog"
]
for bool_col in bool_cols:
    df[bool_col].fillna(0.0, inplace=True)
    df[bool_col].replace("Present", 1.0, inplace=True)

In [ ]:
df

> However, there are also columns that should contain real numbers but which have missing data.

In [ ]:
df.columns[df.isna().any()].tolist()

> Printing the number of non-missing entries, it seems that the columns `fastest5secwinddir` and `fastest5secwindspeed` are not available at all in the dataset, so we drop them. Furthermore, only very few rows have missing values for `snowfall` and `snowdepth`, which we assume to be zero.

In [ ]:
df.count()

In [ ]:
df.drop("fastest5secwinddir", axis=1, inplace=True)
df.drop("fastest5secwindspeed", axis=1, inplace=True)

In [ ]:
missing_cols = ["snowfall", "snowdepth"]
for missing_col in missing_cols:
    df[missing_col].fillna(0.0, inplace=True)

In [ ]:
assert len(df.columns[df.isna().any()].tolist()) == 0

> The main statistics are reported below.

In [ ]:
df.describe()

2. In order to transform the date field in a categorical field, add a new column which stores the season (winter, spring, summer, fall) associated to the date of the row, and drop the date column from the dataset.

*Solution*:
> A simple way to determine the season from a date is to combine the month and the day, in that order, into a single number (e.g., March 12 into 0312 = 312). We then assign a category from 0 to 3, corresponding to each season. The name of the season (at least in the northern hemisphere) is provided as comment. 

In [ ]:
season_col = []
for date in df["date"]:
    if int(date.strftime("%m%d")) >= 1221 or int(date.strftime("%m%d")) <= 319:
        season = 0  # winter
    elif int(date.strftime("%m%d")) >= 320 and int(date.strftime("%m%d")) <= 620:
        season = 1  # spring
    elif int(date.strftime("%m%d")) >= 621 and int(date.strftime("%m%d")) <= 921:
        season = 2  # summer
    elif int(date.strftime("%m%d")) >= 922 and int(date.strftime("%m%d")) <= 1220:
        season = 3  # fall
    season_col.append(season)

df["season"] = season_col
df.drop("date", axis=1, inplace=True)

In [ ]:
df

> We then replace the categorical column with dummy variables, as we already did in tutorial 2.

In [ ]:
dummy_seasons = pd.get_dummies(df["season"], prefix="season")
dummy_seasons.head()

> We exclude the first dummy, and collect the remaining column in the dataset.

In [ ]:
dummy_seasons.drop("season_0", axis=1, inplace=True)
df.drop("season", axis=1, inplace=True)
df = pd.concat([df, dummy_seasons], axis=1)

In [ ]:
df.head()

3. After normalizing the dataset, define the `temperaturemin` column as the observation $\boldsymbol{y}$, and the remaining columns as features $\boldsymbol{X}$.

   Separate the dataset $(\boldsymbol{X}, \boldsymbol{y})$, composed of $m$ rows, in:
    1. a training dataset $(\boldsymbol{X}_{\text{train}}, \boldsymbol{y}_{\text{train}})$, composed of $m_{\text{train}} = 0.8 m$ randomly selected rows, and
    2. a test dataset $(\boldsymbol{X}_{\text{test}}, \boldsymbol{y}_{\text{test}})$, composed of the remaining $m_{\text{test}} = 0.2 m$ rows.
    
*Solution*:
> Normalization can be obtained in `pandas` as follows.

In [ ]:
df = (df - df.min()) / (df.max() - df.min())

In [ ]:
df

> We then split the dataset as requested.

In [ ]:
y = df[["temperaturemin"]].to_numpy().reshape(-1)
y

In [ ]:
X = df.drop("temperaturemin", axis=1).to_numpy()
X

In [ ]:
X.shape

In [ ]:
np.random.seed(42 + 300)
perm = np.random.permutation(y.shape[0])
perm

In [ ]:
m_train = int(0.8 * perm.shape[0])
y_train = y[perm[:m_train]]
X_train = X[perm[:m_train]]
y_test = y[perm[m_train:]]
X_test = X[perm[m_train:]]

4. Implement the evaluation of the empirical risk associated to a linear regression via least squares with multiple features, as well as its gradient.

*Solution*:
> This is very similar to the previous exercise, except that now we have a vector of features. Since here we have 28 features, than $n = 29$ (28 features + 1 bias) The prediction function is $\hat{y}(\boldsymbol{x}; \boldsymbol{w}) = \sum_{i = 1}^{n - 1} w^{(i)} x^{(i)} + w^{(n)}$, where the first $n - 1$ weights are associated to the corresponding feature, while the last weight is a bias.

In [ ]:
def y_hat(x_j: np.ndarray, w: np.ndarray) -> float:
    """Evaluate the prediction function associated to a linear regression."""
    return np.dot(w[:-1], x_j) + w[-1]

> The least squares loss and its derivatives are:
$$\ell(x, y; \boldsymbol{w}) = \left(\sum_{i = 1}^{n - 1} w^{(i)} x^{(i)} + w^{(n)} - y\right)^2.$$
$$\nabla_\boldsymbol{w} \ell(x, y; \boldsymbol{w}) = 2 \left(\sum_{i = 1}^{n - 1} w^{(i)} x^{(i)} + w^{(n)} - y\right) \begin{bmatrix}\boldsymbol{x}\\1\end{bmatrix}.$$

In [ ]:
def least_squares_loss(x_j: np.ndarray, y_j: float, w: np.ndarray) -> float:
    """Evaluate the least squares loss."""
    return (y_hat(x_j, w) - y_j)**2

In [ ]:
def grad_least_squares_loss(x_j: np.ndarray, y_j: float, w: np.ndarray) -> np.ndarray:
    """Evaluate the gradient of the least squares loss."""
    vec = np.zeros(w.shape[0])
    vec[:-1] = x_j
    vec[-1] = 1
    return 2 * (y_hat(x_j, w) - y_j) * vec

> The empirical risk is the obtained summing over all elements of the training dataset.

In [ ]:
def f_ex_4_2(w: np.ndarray, addend: int = None) -> float:
    """
    Evaluate the empirical risk (if addend is None).

    For use within a stochastic method, the optional parameter addend may take an integer value.
    In such case, the loss associated to the addend-th element is computed instead.
    """
    if addend is None:
        m_train = X_train.shape[0]
        return 1 / m_train * sum(least_squares_loss(x_j, y_j, w) for (x_j, y_j) in zip(X_train, y_train))
    else:
        return least_squares_loss(X_train[addend], y_train[addend], w)

In [ ]:
def grad_f_ex_4_2(w: np.ndarray, addend: int = None) -> np.ndarray:
    """
    Evaluate the gradient of the empirical risk (if addend is None).

    For use within a stochastic method, the optional parameter addend may take an integer value.
    In such case, the gradient loss associated to the addend-th element is computed instead.
    """
    if addend is None:
        m_train = X_train.shape[0]
        return 1 / m_train * sum(grad_least_squares_loss(x_j, y_j, w) for (x_j, y_j) in zip(X_train, y_train))
    else:
        return grad_least_squares_loss(X_train[addend], y_train[addend], w)

5. Train the linear regression model using a mini-batch stochastic gradient descent, with mini-batch size equal to $10\%, 20\%, 50\%$ or $100\%$ of the training set cardinality.

*Solution*:
> We first copy the implementation of the minibatch stochastic gradient from the previous tutorial.

In [ ]:
def mini_batch_stochastic_gradient(
    m: int, m_b: int, f: typing.Callable, grad_f: typing.Callable, alpha: float, epsilon: float,
    maxit: int, w_0: np.ndarray
) -> typing.Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Run the mini-batch stochastic gradient method with constant step length.

    Parameters
    ----------
    m : int
        number of addends in the expression of the cost function.
    m_b : int
        size of the mini-batch.
    f, grad_f : Python function
        callable evaluating the cost function and its gradient, respectively.
    alpha : float
        constant step length.
    epsilon : float
        tolerance for the stopping criterion on the error on the norm of the gradient of the cost.
    maxit : int
        maximum number of allowed iterations.
    w_0 : 1d numpy array
        numpy array containing the initial condition.

    Returns
    -------
    2d numpy array
        history of the optimization variables iterations.
    1d numpy array
        history of the cost function values.
    2d numpy array
        history of the gradient of the cost function.
    """
    # Prepare lists collecting the required outputs over the iterations
    all_w = [w_0]
    all_f = [f(w_0)]
    all_grad_f = [grad_f(w_0)]

    # Prepare iteration counter
    k = 0

    # Use the norm of the gradient as stopping criterion.
    while np.linalg.norm(all_grad_f[k]) > epsilon:
        w_k = all_w[k]

        # Draw random indices
        J_k = np.random.choice(m, size=m_b, replace=False)

        # Compute the update direction
        g_k = - 1 / m_b * sum([grad_f(w_k, addend=j) for j in J_k])

        # Compute w_{k + 1}
        w_k_plus_1 = w_k + alpha * g_k

        # Update required outputs
        all_w.append(w_k_plus_1)
        all_f.append(f(w_k_plus_1))
        all_grad_f.append(grad_f(w_k_plus_1))

        # Increment iteration counter
        k += 1

        # Bail out if exceeded allowed number of iterations
        if k >= maxit:
            print("WARNING: stochastic gradient method exceeded number of allowed iterations")
            break

    # For convenience we transform the outputs into numpy array before returning
    return np.array(all_w), np.array(all_f), np.array(all_grad_f)

> We compute the smoothness constant $L$ in order to determine an appropriate step size $1/L$, defined for a linear regression problem as
$$L = 2 \frac{\lambda_{\max}(\boldsymbol{X}_{\text{train}}^T \boldsymbol{X}_{\text{train}})}{m_{\text{train}}}.$$

In [ ]:
eigs, _ = np.linalg.eig(np.dot(X_train.T, X_train))
L = 2 * np.max(eigs) / m_train
print("L =", L)

> We then train the model for different mini-batch sizes. We print the total number of iterations $K$, but also the number of epochs, defined as
$$E = K \frac{m_b}{m}$$
and the elapsed time. Notice how training with a small mini-batch size takes a very similar number of iterations than training with the whole gradient. However, since the dataset is composed of thousands of rows, a mini-batch iterations is far less expensive than one interation of the gradient method, thus mini-batch methods (even though stochastic) may even be faster than the deterministic gradient method! This is the first example we see that shows how efficient stochastic methods are on real datasets, which is part of the reason why they are so successful in machine learning.

In [ ]:
import time

In [ ]:
all_w = dict()
all_f = dict()
all_grad_f = dict()

np.random.seed(42 + 500)

all_perc = [10, 20, 50, 100]
for perc in all_perc:
    m_b = int(perc / 100 * m_train)
    start = time.time()
    all_w[perc], all_f[perc], all_grad_f[perc] = mini_batch_stochastic_gradient(
        m_train, m_b, f_ex_4_2, grad_f_ex_4_2, 1 / L, 1e-2, 200, np.zeros(X.shape[1] + 1))
    end = time.time()
    K = all_w[perc].shape[0]
    E = int(np.ceil(K * m_b / m_train))
    print(
        "m_b =", m_b, "out of", m_train, "converged in", K, "iterations,",
        E, "epochs and", end - start, "seconds")

6. Evaluate the accuracy of the prediction, and determine which are the most relevant features while carrying out the prediction task.

*Solution*:
> We evaluate the root mean squared error (RMSE) for training and test set, for all configurations of mini-batch size.

In [ ]:
for perc in all_perc:
    RMSE_train = np.sqrt(
        1 / X_train.shape[0]
        * sum((y_hat(x_j, all_w[perc][-1]) - y_j)**2 for (x_j, y_j) in zip(X_train, y_train)))
    RMSE_test = np.sqrt(
        1 / X_test.shape[0]
        * sum((y_hat(x_j, all_w[perc][-1]) - y_j)**2 for (x_j, y_j) in zip(X_test, y_test)))
    m_b = int(perc / 100 * m_train)
    print("m_b =", m_b, "out of", m_train, "RMSE_train =", RMSE_train, "RMSE_test =", RMSE_test)

> We notice that in all cases the RMSE is of the of the order of $10\%$. For our goals this prediction is reasonably accurate. 

> In order to determine which are the most relevant features, we look at the largest entries among the weights.

In [ ]:
all_w[100][-1]

> We notice that there are a few coefficients which are of $O(10^{-1})$, and many that are of lower order of magnitude. In particular

In [ ]:
important_features = np.argwhere(np.abs(all_w[100][-1]) > 0.1)
important_features

> To get the corresponding feature names, we query the `pandas` dataframe. Notice how we should drop the last entry from `important_features`, because it is the coefficient associated to the bias (and not to a feature).

In [ ]:
[df.drop("temperaturemin", axis=1).columns[f][0] for f in important_features[:-1]]

> We could have probably guessed that these were the important features just by looking at the Pearson correlation coefficient in the dataset. Notice how all the features selected as important are at the very top of the list of such correlation.

In [ ]:
correlation = df.corr()["temperaturemin"]
correlation.sort_values(ascending=False, key=abs)

> Is there a way to change the optimization problem used during training so that it automatically discards features (i.e., sets a zero weight in front of them) which are not relevant in the prediction of the output?

## Exercise 4.3 (continuation of Exercise 4.1)

Consider a scalar feature $x$ in the domain domain $I = [-1, 1]$ and the observation $y$ defined as
$$y = g(x) + \xi,$$
where the (deterministic) function $g$ is defined as
$$g(x) = \cos(1.5 \pi x)$$
and $\xi$ is a white Gaussian noise with zero mean and variance $\sigma^2 = 0.01$.

5. Re-generate the same training dataset $\boldsymbol{x}_{\text{train}} \in \mathbb{R}^{200}, \boldsymbol{y}_{\text{train}} \in \mathbb{R}^{200}$ and test dataset $\boldsymbol{x}_{\text{test}} \in \mathbb{R}^{50}, \boldsymbol{y}_{\text{test}} \in \mathbb{R}^{50}$ that was used in Exercise 4.1.

*Solution*:
> We copy the code from Exercise 4.1. Notice that, in order to generate the *same* dataset, it is important to set the same seed that we used in Exercise 4.1.

In [ ]:
def g(x: np.ndarray) -> np.ndarray:
    """Evaluate g(x)."""
    return np.cos(1.5 * np.pi * x)

In [ ]:
np.random.seed(41)
x_train = np.random.uniform(-1, 1, size=30)
x_test = np.random.uniform(-1, 1, size=50)
y_train = g(x_train) + np.random.normal(0, np.sqrt(0.01), size=30)
y_test = g(x_test) + np.random.normal(0, np.sqrt(0.01), size=50)

6. Implement the evaluation of the empirical risk associated to a polynomial regression via least squares with ridge regression, as well as its gradient.

*Solution*:
> The prediction function is the same as Exercise 4.1.

In [ ]:
def y_hat(x_j: float, w: np.ndarray) -> float:
    """Evaluate the prediction function associated to a polynomial regression."""
    return sum(w[i] * x_j**i for i in range(w.shape[0]))

> The least squares loss with $\ell^2$ regularization and its derivatives are:
$$\ell_{\text{reg}}(x, y; \boldsymbol{w}) = \left(\sum_{i = 0}^{n - 1} w^{(i)} x^i - y\right)^2 + \lambda \sum_{i = 0}^{n - 1} \left[w^{(i)}\right]^2.$$
$$\nabla_\boldsymbol{w} \ell_{\text{reg}}(x, y; \boldsymbol{w}) = 2 \left(\sum_{i = 0}^{n - 1} w^{(i)} x^i - y\right) \begin{bmatrix}1\\x\\x^2\\\dots\\x^{n-1}\end{bmatrix} + 2 \lambda \boldsymbol{w}.$$

In [ ]:
def least_squares_loss_reg(x_j: float, y_j: float, w: np.ndarray) -> float:
    """Evaluate the least squares loss with ridge penalization term."""
    return (y_hat(x_j, w) - y_j)**2 + lambda_ * np.linalg.norm(w)**2

In [ ]:
def grad_least_squares_loss_reg(x_j: float, y_j: float, w: np.ndarray) -> np.ndarray:
    """Evaluate the gradient of the least squares loss with ridge penalization term."""
    powers = np.arange(w.shape[0])
    return 2 * (y_hat(x_j, w) - y_j) * x_j**powers + 2 * lambda_ * w

> The empirical risk is then definied by summing the regularized loss over the dataset.

In [ ]:
def f_ex_4_3(w: np.ndarray) -> float:
    """Evaluate the empirical risk."""
    m_train = x_train.shape[0]
    return 1 / m_train * sum(least_squares_loss_reg(x_j, y_j, w) for (x_j, y_j) in zip(x_train, y_train))

In [ ]:
def grad_f_ex_4_3(w: np.ndarray) -> np.ndarray:
    """Evaluate the gradient of the empirical risk."""
    m_train = x_train.shape[0]
    return 1 / m_train * sum(grad_least_squares_loss_reg(x_j, y_j, w) for (x_j, y_j) in zip(x_train, y_train))

7. Choose $\varepsilon = 10^{-8}$, $\boldsymbol{w}_0 = \boldsymbol{0}$ and $h_0 = 1$. Obtain the best fit polynomial for $n$ between 1 and 25 by running BFGS, and $\lambda \in \{10^{-1}, 10^{-5}, 10^{-9}\}$. Comment on the quality of the fit when $n$ increases.

*Solution*:
> We first copy the BFGS implementation we used in Exercise 4.1.

In [ ]:
def bfgs(
    f: typing.Callable, grad_f: typing.Callable, alpha: float, epsilon: float, maxit: int, w_0: np.ndarray,
    h_0: float
) -> typing.Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Run BFGS method with constant step length.

    Parameters
    ----------
    f, grad_f : Python function
        callable evaluating the cost function and its gradient, respectively.
    alpha : float
        constant step length.
    c_1, c_2 : float
        constants of the backtracking algorithm.
    epsilon : float
        tolerance for the stopping criterion on the error on the norm of the gradient of the cost.
    maxit : int
        maximum number of allowed iterations.
    w_0 : 1d numpy array
        numpy array containing the initial condition.
    h_0 : float
        diagonal scaling factor that defines the initial approximation of the inverse of the hessian matrix.

    Returns
    -------
    2d numpy array
        history of the optimization variables iterations.
    1d numpy array
        history of the cost function values.
    2d numpy array
        history of the gradient of the cost function.
    """
    # Prepare lists collecting the required outputs over the iterations
    all_w = [w_0]
    all_f = [f(w_0)]
    all_grad_f = [grad_f(w_0)]

    # Prepare iteration counter
    k = 0

    # Prepare initial approximation of inverse of the hessian: take the identity matrix
    I = np.eye(w_0.shape[0])
    inv_hess_f_k = h_0 * I

    # Use the norm of the gradient as stopping criterion.
    while np.linalg.norm(all_grad_f[k]) > epsilon:
        w_k = all_w[k]
        grad_f_k = all_grad_f[k]

        # Compute w_{k+1}
        w_k_plus_1 = w_k - alpha * np.dot(inv_hess_f_k, grad_f_k)

        # Update required outputs
        all_w.append(w_k_plus_1)
        all_f.append(f(w_k_plus_1))
        all_grad_f.append(grad_f(w_k_plus_1))

        # Update approximation of inverse of the hessian
        y_k = all_grad_f[k + 1] - all_grad_f[k]
        s_k = all_w[k + 1] - all_w[k]
        rho_k = 1 / np.dot(y_k, s_k)
        inv_hess_f_k = (
            np.dot(np.dot(I - rho_k * np.outer(s_k, y_k), inv_hess_f_k), I - rho_k * np.outer(y_k, s_k))
            + rho_k * np.outer(s_k, s_k)
        )

        # Increment iteration counter
        k += 1

        # Bail out if exceeded allowed number of iterations
        if k >= maxit:
            print("WARNING: BFGS method exceeded number of allowed iterations")
            break

    # For convenience we transform the outputs into numpy array before returning
    return np.array(all_w), np.array(all_f), np.array(all_grad_f)

> We run the training for all required values of $n$ and $\lambda$.

In [ ]:
all_lambdas = [1e-1, 1e-5, 1e-9]

all_w = {lambda_: dict() for lambda_ in all_lambdas}
all_f = {lambda_: dict() for lambda_ in all_lambdas}
all_grad_f = {lambda_: dict() for lambda_ in all_lambdas}

for lambda_ in all_lambdas:
    # Note that the variable lambda_ will then be used in the functions least_squares_loss_reg
    # and grad_least_squares_loss_reg, which are called in f_ex_4_3 and grad_f_ex_4_3
    for n in range(1, 26):
        all_w[lambda_][n], all_f[lambda_][n], all_grad_f[lambda_][n] = bfgs(
            f_ex_4_3, grad_f_ex_4_3, 1, 1e-8, 1000, np.zeros(n), 1)

> We then create three plots, one for each value of $\lambda$. Each plot has two rows, corresponding to the training set (top) and the test set (bottom), and four columns, each column corresponding to a value of $n \in \{3, 7, 13, 25\}$.

In [ ]:
x_plot = np.linspace(-1, 1, 500)

In [ ]:
n_plot = [3, 7, 13, 25]
for lambda_ in all_lambdas:
    fig = plotly.subplots.make_subplots(rows=2, cols=4, vertical_spacing=0.05, horizontal_spacing=0.05)
    for (col, n) in enumerate(n_plot):
        fig.add_scatter(
            x=x_train, y=y_train,
            marker=dict(color=plotly.colors.qualitative.Set1[0], size=6),
            mode="markers", name="training dataset",
            row=1, col=col + 1, showlegend=(col == 0)
        )
        fig.add_scatter(
            x=x_test, y=y_test,
            marker=dict(color=plotly.colors.qualitative.Set1[1], size=6),
            mode="markers", name="test dataset",
            row=2, col=col + 1, showlegend=(col == 0)
        )
        fig.add_scatter(
            x=x_plot, y=y_hat(x_plot, all_w[lambda_][n][-1]),
            line=dict(color=plotly.colors.qualitative.Set1[6], width=2),
            mode="lines", name="y_hat(x)",
            row=1, col=col + 1, showlegend=(col == 0)
        )
        fig.add_scatter(
            x=x_plot, y=y_hat(x_plot, all_w[lambda_][n][-1]),
            line=dict(color=plotly.colors.qualitative.Set1[6], width=2),
            mode="lines", name="y_hat(x)",
            row=2, col=col + 1, showlegend=False
        )
    fig.update_layout(
        title=(
            "lambda = " + str(lambda_) + ". Training dataset (top) and test dataset (bottom). "
            + "Increasing n = " + str(n_plot) + " (left to right)"
        ), width=2 * 512, height=512, autosize=False
    )
    # fig.update_yaxes(range=[-1.2, 1.2])
    fig.show()

> * From the first plot we see that the regularization coefficient $\lambda = 10^{-1}$ is too large. The regularization term is dominating the overall empirical risk, and this prevents a good fitting to the data.
> * From the third plot we see that the regularization coefficient $\lambda = 10^{-9}$ is too small. Indeed, the prediction for large values of $n$ is still oscillating, especially close to the boundaries of the interval $I$.
> * From the second plot we realize that the choice $\lambda = 10^{-5}$ is the best one among the proposed values for the regularization coefficient. Indeed, the fitting of the training dataset and the prediction on the test dataset look as good as the unregularized case in the second column ($n = 7$), and oscillations do not affect too much the prediction curve in the last two columns of the plot (especially the third column, $n = 13$).
>
> To get a further qualitative comparison between the proposed values of $\lambda$, we prepare three RMSE plots as well.

In [ ]:
RMSE_train = {
    lambda_: np.array([
        np.sqrt(1 / x_train.shape[0] * np.sum((y_hat(x_train, all_w[lambda_][n][-1]) - y_train)**2))
        for n in range(1, 26)
    ]) for lambda_ in all_lambdas
}
RMSE_test = {
    lambda_: np.array([
        np.sqrt(1 / x_test.shape[0] * np.sum((y_hat(x_test, all_w[lambda_][n][-1]) - y_test)**2))
        for n in range(1, 26)
    ]) for lambda_ in all_lambdas
}

In [ ]:
for lambda_ in all_lambdas:
    fig = go.Figure()
    fig.add_scatter(x=np.arange(RMSE_train[lambda_].shape[0]) + 1, y=RMSE_train[lambda_], name="Training set")
    fig.add_scatter(x=np.arange(RMSE_test[lambda_].shape[0]) + 1, y=RMSE_test[lambda_], name="Test set")
    fig.update_layout(title="lambda = " + str(lambda_) + ". RMSE")
    fig.update_yaxes(type="log", exponentformat="power")
    fig.show()

> * From the first plot we confirm that the regularization coefficient $\lambda = 10^{-1}$ is too large. Indeed, RMSE for both training and test sets are considerably larger than the best value we had obtained in the unregularized case.
> * From the third plot we confirm that the regularization coefficient $\lambda = 10^{-9}$ is too small. Indeed, RMSE for both training and test sets looks very close to the curve we had obtained in the unregularized case, meaning that the regularization has a very minor effect on the choice of the weights.
> * From the second plot we confirm that the choice $\lambda = 10^{-5}$ is the best one among the proposed values. Indeed, the best value of the RMSE on the test dataset is comparable to the one we had obtained in the unregularized case, and the increase of RMSE on the test dataset when $n$ increases is very moderate compared to the steep increase detected in the unregularized case.
>
> In order to better understand how ridge regularization is allowing to improve the results, we print for $n$ above 9 a comparison of the weights between the best regularized case $\lambda = 10^{-5}$ (first row of the print) and the case $\lambda = 10^{-9}$ (second row of the print), which is almost an unregularized regression.

In [ ]:
for n in range(10, 26):
    print("n =", n)
    weights_comparison = np.vstack([all_w[lambda_][n][-1] for lambda_ in all_lambdas[1:]])
    with np.printoptions(precision=2, suppress=True):
        print(weights_comparison)
    print()

> We notice that, for each fixed $n$, the regularization has the effect of (considerably, in some cases) decreasing the values of the weights $w^{(i)}$ associated to indices where $i$ is big (i.e., high monomial degree $x^i$). Notice however that, even tough decreased, such values are still non-zero, so ridge regression *did not* operate a feature selection. We will see next an exercise on feature selection.
>
> *Bonus remark*: notice how, in the regularized case, the weights in even positions (numbering positions from zero as Python does, so positions 0, 2, 4, ...) are typically larger in absolute value then those in odd positions (positions 1, 3, 5, ...). Looking at the expression (or the plot) of the function $g$, can you give a mathematical explanation of why this happens?

## Exercise 4.4 (continuation of Exercise 4.2)

The US National Centers for Environmental Information has collected weather data at the Raleigh Durham International Airport, that we are using to prediction the lowest temperature in any given day.

7. Re-generate the same training dataset $\boldsymbol{X}_{\text{train}}, \boldsymbol{y}_{\text{train}}$ and test dataset $\boldsymbol{x}_{\text{test}}, \boldsymbol{y}_{\text{test}}$ that was used in Exercise 4.2.

*Solution*:
> We re-import the data using `pandas`, and carry out the same preprocessing operations that we were required to do in Exercise 4.2.

In [ ]:
import os

In [ ]:
import pandas as pd

In [ ]:
if os.path.isfile("data/rdu-weather-history.csv"):
    csv_path = "data/rdu-weather-history.csv"
else:
    # csv_path = "https://data.townofcary.org/explore/dataset/rdu-weather-history/download/?format=csv"
    csv_path = (
        "https://dmf.unicatt.it/~fball/public/optimization_for_machine_learning"
        + "/rdu-weather-history.csv"
    )
df = pd.read_csv(csv_path, parse_dates=["date"], sep=";")

In [ ]:
bool_cols = [
    "fog", "fogheavy", "mist", "rain", "fogground", "ice", "glaze", "drizzle", "snow", "freezingrain",
    "smokehaze", "thunder", "highwind", "hail", "blowingsnow", "dust", "freezingfog"
]
for bool_col in bool_cols:
    df[bool_col].fillna(0.0, inplace=True)
    df[bool_col].replace("Present", 1.0, inplace=True)
df.drop("fastest5secwinddir", axis=1, inplace=True)
df.drop("fastest5secwindspeed", axis=1, inplace=True)
missing_cols = ["snowfall", "snowdepth"]
for missing_col in missing_cols:
    df[missing_col].fillna(0.0, inplace=True)
assert len(df.columns[df.isna().any()].tolist()) == 0

In [ ]:
season_col = []
for date in df["date"]:
    if int(date.strftime("%m%d")) >= 1221 or int(date.strftime("%m%d")) <= 319:
        season = 0  # winter
    elif int(date.strftime("%m%d")) >= 320 and int(date.strftime("%m%d")) <= 620:
        season = 1  # spring
    elif int(date.strftime("%m%d")) >= 621 and int(date.strftime("%m%d")) <= 921:
        season = 2  # summer
    elif int(date.strftime("%m%d")) >= 922 and int(date.strftime("%m%d")) <= 1220:
        season = 3  # fall
    season_col.append(season)

df["season"] = season_col
df.drop("date", axis=1, inplace=True)
dummy_seasons = pd.get_dummies(df["season"], prefix="season")
dummy_seasons.drop("season_0", axis=1, inplace=True)
df.drop("season", axis=1, inplace=True)
df = pd.concat([df, dummy_seasons], axis=1)

In [ ]:
df = (df - df.min()) / (df.max() - df.min())

In [ ]:
y = df[["temperaturemin"]].to_numpy().reshape(-1)
X = df.drop("temperaturemin", axis=1).to_numpy()
np.random.seed(42 + 300)
perm = np.random.permutation(y.shape[0])
m_train = int(0.8 * perm.shape[0])
y_train = y[perm[:m_train]]
X_train = X[perm[:m_train]]
y_test = y[perm[m_train:]]
X_test = X[perm[m_train:]]

8. Implement the ISTA algorithm with mini-batch stochastic gradient computation and with constant step length in a Python function. Such function should:
   * take as input the number $m$ of addends, the size $m_b$ of a mini-batch, the (unregularized) function $s$, its gradient $\nabla s$, the regularization coefficient $\lambda$, the value $\alpha$ of the step length, the tolerance $\varepsilon$ for the stopping criterion, maximum number $K_{\max}$ of allowed iterations, and the initial condition $\boldsymbol{w}_{0}$;
   * return as outputs the optimization variable iterations $\{\boldsymbol{w}_k\}_k$, the corresponding function values $\{s(\boldsymbol{w}_k)\}_k$ and gradients $\{\nabla s(\boldsymbol{w}_k)\}_k$.

   Use the stopping criterion based on the norm of the gradient (of the unregularized function $s$).
 
*Solution*:
> We start defining the soft thresholding function. Since we are interested in applying it to a vector $\boldsymbol{w}$ (and not a scalar component $w^{(i)}$), we implement it using operators `<` and `>` applied elementwise, rather than `if` cases.

In [ ]:
def soft_thresholding(w: np.ndarray, eta: float) -> np.ndarray:
    """Apply the soft thresholding function elementwise."""
    return (w + eta) * (w < - eta) + (w - eta) * (w > eta)

> Check the elementwise application on a simple case.

In [ ]:
soft_thresholding(np.array([-10, -5, 5, 10]), eta=7)

> We next implement the mini-batch version of ISTA. It is enough to copy the implementation of `mini_batch_stochastic_gradient` and only change the line that computes $\boldsymbol{w}_{k+1}$ in order to apply the soft thresholding operator.

In [ ]:
def mini_batch_ista(
    m: int, m_b: int, s: typing.Callable, grad_s: typing.Callable, lambda_: float, alpha: float, epsilon: float,
    maxit: int, w_0: np.ndarray
) -> typing.Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Run the mini-batch stochastic gradient method with constant step length.

    Parameters
    ----------
    m : int
        number of addends in the expression of the (unregularized) cost function.
    m_b : int
        size of the mini-batch.
    s, grad_s : Python function
        callable evaluating the (unregularized) cost function and its gradient, respectively.
    lambda_ : float
        regularization coefficient.
    alpha : float
        constant step length.
    epsilon : float
        tolerance for the stopping criterion on the error on the norm of the gradient of the cost.
    maxit : int
        maximum number of allowed iterations.
    w_0 : 1d numpy array
        numpy array containing the initial condition.

    Returns
    -------
    2d numpy array
        history of the optimization variables iterations.
    1d numpy array
        history of the cost function values.
    2d numpy array
        history of the gradient of the cost function.
    """
    # Prepare lists collecting the required outputs over the iterations
    all_w = [w_0]
    all_s = [s(w_0)]
    all_grad_s = [grad_s(w_0)]

    # Prepare iteration counter
    k = 0

    # Use the norm of the gradient as stopping criterion.
    while np.linalg.norm(all_grad_s[k]) > epsilon:
        w_k = all_w[k]

        # Draw random indices
        J_k = np.random.choice(m, size=m_b, replace=False)

        # Compute the update direction
        g_k = - 1 / m_b * sum([grad_s(w_k, addend=j) for j in J_k])

        # Compute w_{k + 1}
        w_k_plus_1 = soft_thresholding(w_k + alpha * g_k, alpha * lambda_)

        # Update required outputs
        all_w.append(w_k_plus_1)
        all_s.append(s(w_k_plus_1))
        all_grad_s.append(grad_s(w_k_plus_1))

        # Increment iteration counter
        k += 1

        # Bail out if exceeded allowed number of iterations
        if k >= maxit:
            print("WARNING: ISTA method exceeded number of allowed iterations")
            break

    # For convenience we transform the outputs into numpy array before returning
    return np.array(all_w), np.array(all_s), np.array(all_grad_s)

9. Implement the evaluation of the empirical risk associated to a linear regression via least squares with multiple features, as well as its gradient.

*Solution*:
> We copy here the functions we have implemented in Exercise 4.2 as they are. Note that (in contrast to the changes we had to do when copying from Exercise 4.1 to Exercise 4.3) here there are no changes, because the regularization term $\lambda \left\|\boldsymbol{w}\right\|_1$ is handled separately by the ISTA algorithm!

In [ ]:
def y_hat(x_j: np.ndarray, w: np.ndarray) -> float:
    """Evaluate the prediction function associated to a linear regression."""
    return np.dot(w[:-1], x_j) + w[-1]

In [ ]:
def least_squares_loss(x_j: np.ndarray, y_j: float, w: np.ndarray) -> float:
    """Evaluate the least squares loss."""
    return (y_hat(x_j, w) - y_j)**2

In [ ]:
def grad_least_squares_loss(x_j: np.ndarray, y_j: float, w: np.ndarray) -> np.ndarray:
    """Evaluate the gradient of the least squares loss."""
    vec = np.zeros(w.shape[0])
    vec[:-1] = x_j
    vec[-1] = 1
    return 2 * (y_hat(x_j, w) - y_j) * vec

In [ ]:
def s_ex_4_4(w: np.ndarray, addend: int = None) -> float:
    """
    Evaluate the empirical risk (if addend is None).

    For use within a stochastic method, the optional parameter addend may take an integer value.
    In such case, the loss associated to the addend-th element is computed instead.
    """
    if addend is None:
        m_train = X_train.shape[0]
        return 1 / m_train * sum(least_squares_loss(x_j, y_j, w) for (x_j, y_j) in zip(X_train, y_train))
    else:
        return least_squares_loss(X_train[addend], y_train[addend], w)

In [ ]:
def grad_s_ex_4_4(w: np.ndarray, addend: int = None) -> np.ndarray:
    """
    Evaluate the gradient of the empirical risk (if addend is None).

    For use within a stochastic method, the optional parameter addend may take an integer value.
    In such case, the gradient loss associated to the addend-th element is computed instead.
    """
    if addend is None:
        m_train = X_train.shape[0]
        return 1 / m_train * sum(grad_least_squares_loss(x_j, y_j, w) for (x_j, y_j) in zip(X_train, y_train))
    else:
        return grad_least_squares_loss(X_train[addend], y_train[addend], w)

10. Train the linear regression model using a mini-batch ISTA algorithm, varying:
    * mini-batch size, equal to $10\%, 20\%, 50\%$ or $100\%$ of the training set cardinality, and
    * regularization coefficient, equal to $10^{-1}$, $10^{-3}$ or $10^{-5}$.
 
*Solution*:
> We recompute the value of the smoothness constant $L$, to be used in the evaluation of the step size $\alpha = 1 / L$.

In [ ]:
eigs, _ = np.linalg.eig(np.dot(X_train.T, X_train))
L = 2 * np.max(eigs) / m_train
print("L =", L)

> We next train the regression model for all requested values of $m_b$ and $\lambda$.

In [ ]:
all_lambdas = [1e-1, 1e-3, 1e-5]
all_perc = [10, 20, 50, 100]

all_w = {lambda_: dict() for lambda_ in all_lambdas}
all_s = {lambda_: dict() for lambda_ in all_lambdas}
all_grad_s = {lambda_: dict() for lambda_ in all_lambdas}

np.random.seed(44 + 1000)

for lambda_ in all_lambdas:
    print("lambda =", lambda_)
    for perc in all_perc:
        m_b = int(perc / 100 * m_train)
        all_w[lambda_][perc], all_s[lambda_][perc], all_grad_s[lambda_][perc] = mini_batch_ista(
            m_train, m_b, s_ex_4_4, grad_s_ex_4_4, lambda_, 1 / L, 1e-2, 200, np.zeros(X.shape[1] + 1))
        K = all_w[lambda_][perc].shape[0]
        E = int(np.ceil(K * m_b / m_train))
        print("\t m_b =", m_b, "out of", m_train, "converged in", K, "iterations,", E, "epochs")
        non_zero = np.count_nonzero(all_w[lambda_][perc][-1]) / all_w[lambda_][perc][-1].shape[0]
        print("\t percentage of non zero coefficients:", non_zero)

> * When $\lambda = 10^{-1}$ the optimization did not converge in the required number of iterations. We also see that the feature selection is very aggressive, often discarding more than $90\%$ of the available features.
> * When $\lambda = 10^{-3}$ we do get convergence to models that often discard around $40\%$ of the available features.
> * When $\lambda = 10^{-5}$ no feature selection is happening, therefore we expect to obtain a model which is very similar to the unregularized one.

11. Evaluate the accuracy of the prediction, and determine which are the most relevant features while carrying out the prediction task. Are they the same as in the unregularized case?

*Solution*:
> We compute the RMSE on the training and test sets for all values of $m_b$ and $\lambda$.

In [ ]:
for lambda_ in all_lambdas:
    print("lambda =", lambda_)
    for perc in all_perc:
        RMSE_train = np.sqrt(
            1 / X_train.shape[0]
            * sum((y_hat(x_j, all_w[lambda_][perc][-1]) - y_j)**2 for (x_j, y_j) in zip(X_train, y_train)))
        RMSE_test = np.sqrt(
            1 / X_test.shape[0]
            * sum((y_hat(x_j, all_w[lambda_][perc][-1]) - y_j)**2 for (x_j, y_j) in zip(X_test, y_test)))
        m_b = int(perc / 100 * m_train)
        print("\t m_b =", m_b, "out of", m_train, "RMSE_train =", RMSE_train, "RMSE_test =", RMSE_test)

> * The case $\lambda = 10^{-1}$ is giving the worst RMSE performance. We are thus led to believe that, by being too aggressive with feature selection, we have pruned some features that were actually important!
> * The cases $\lambda = 10^{-3}$ and $\lambda = 10^{-5}$ have very similar RMSE results. If interpretability of the prediction model is important, then we suggest to use the model resulting from $\lambda = 10^{-3}$ because it is easier to read, having discarded around $40\%$ of the available features.
>
> Finally, we conclude listing which are the important features of each model (i.e., following what we did in Exercise 4.2, the features with weight greater than 0.1).

In [ ]:
for lambda_ in all_lambdas:
    print("lambda =", lambda_)
    for perc in all_perc:
        important_features = np.argwhere(np.abs(all_w[lambda_][perc][-1]) > 0.1)
        names = [df.drop("temperaturemin", axis=1).columns[f][0] for f in important_features[:-1]]
        m_b = int(perc / 100 * m_train)
        print("\t m_b =", m_b, "out of", m_train, "important features:", names)

> * The models trained with $\lambda = 10^{-3}$ and $\lambda = 10^{-5}$ agree with the unregularized training in determining which features have been detected as important.
> * In contrast, the model trained with $\lambda = 10^{-1}$ has discarded (or classified as unimportant) the two seasonal features. This confirms our intuition that with such a large value of $\lambda$ important features were pruned, leading to a worse predictive model.